# Dataset Reference Extraction

This notebook demonstrates how to use the `DatasetReferenceExtractor` class and regex patterns 
to extract dataset references from PDF and XML files.  

### Setup

Import required libraries and configure settings for Pandas and warnings.


In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Load Extractor and Regex Patterns

We use the custom `DatasetReferenceExtractor` class with a predefined set of regex patterns 
for identifiers like DOIs, BioProject IDs, UniProt IDs, GEO accessions, etc.


### Run Extraction

Apply the extractor on a folder of PDFs and XMLs.  
The output is a DataFrame with extracted references and their surrounding context.


In [4]:
from dataset_reference_extractor import DatasetReferenceExtractor
from regex_patterns import patterns

extractor = DatasetReferenceExtractor(regex_patterns=patterns, context_window=40)

df = extractor.run_on_folder(
    pdf_folder="data/train/PDF",
    xml_folder="data/train/XML"
)

df.head()

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: uns

,row_id,article_id,dataset_id,context,pattern_type,source,start_idx,score
0,0,10.1002_2017jc013030,https://doi.org/10.3354/meps11580,properties of some phytoplankters.Deep Sea Res...,doi_short,pdf,58307,0
1,1,10.1002_2017jc013030,https://doi.org/10.4319/lo.2011.56.3.0955,southern Bay of Biscay .Marine Ecology Progres...,doi_short,pdf,58569,0
2,2,10.1002_2017jc013030,https://doi.org/10.4319/lo.2003.48.2.0843,bio-optical oceanic regimes.Limnology and Ocea...,doi_short,pdf,58828,0
3,3,10.1002_2017jc013030,https://doi.org/10.1016/s0967-0637,J Nolan 10.1016/S0967-0637 01 00025-5 Deep Sea...,doi_xml,xml,57177,0
4,4,10.1002_2017jc013030,https://doi.org/10.4319/lo.1996.41.8.1684,Part I: Oceanographic Research Papers 48 11 24...,doi_short,pdf,59686,0


### Notes
- This notebook focuses on **regex-based extraction** only.  
- Planned NLP-based scoring and classification (usage vs. citation vs. access) 
  were dropped due to time/scope.  